# 의사결정트리 완벽 프레임 v2 - 시험용 🌳
## 모든 기출 유형 대응! (F1 Score, ROC AUC, Accuracy 모두 포함)

## 1. 필수 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

# dmba 라이브러리 (있으면)
try:
    from dmba import classificationSummary
    HAS_DMBA = True
except:
    HAS_DMBA = False
    print("dmba 라이브러리가 없습니다. 기본 지표만 사용합니다.")

## 2. 데이터 로드 및 탐색

In [ ]:
# 데이터 로드 (⭐ 파일명 수정 필수!)
df = pd.read_csv('데이터파일.csv')

print("[데이터 정보]")
df.info()

print("\n[결측치 확인]")
print(df.isnull().sum())

print("\n[데이터 샘플]")
print(df.head())

In [ ]:
# 변수 타입 자동 분류
print("[변수 타입 분류]")
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"범주형 변수 ({len(categorical_cols)}개): {categorical_cols}")
print(f"수치형 변수 ({len(numerical_cols)}개): {numerical_cols}")

## 3. 데이터 전처리

In [ ]:
# 결측치 제거 (기출에서 자주 요구)
print(f"결측치 제거 전: {df.shape}")
df = df.dropna()
print(f"결측치 제거 후: {df.shape}")

# 목표변수를 범주형으로 변환 (필요시)
# df['목표변수'] = df['목표변수'].astype('category')

# 독립변수/종속변수 분리 (⭐ 여기 수정 필수!)
outcome = '목표변수'  # ← 종속변수 이름 입력
predictors = [col for col in df.columns 
              if col not in [outcome, '제외할변수1', '제외할변수2']]

X = df[predictors]
y = df[outcome]

# 원-핫 인코딩 (의사결정트리는 drop_first=False 가능)
X = pd.get_dummies(X, drop_first=False)

print(f"\n독립변수 개수: {X.shape[1]}")
print(f"데이터 크기: {X.shape}")
print(f"변수 목록 (처음 10개): {X.columns.tolist()[:10]}")
print(f"\n목표변수 분포:\n{y.value_counts()}")

## 4. 데이터 분리

In [ ]:
# 방법 1: train_test_split 사용 (일반적)
train_X, valid_X, train_y, valid_y = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y
)

print(f"학습 데이터: {train_X.shape}")
print(f"검증 데이터: {valid_X.shape}")
print(f"\n학습 데이터 클래스 분포:\n{train_y.value_counts()}")

In [ ]:
# 방법 2: 인덱스 기반 분리 (기출문제 스타일)
# 예: "0~1999번까지 학습, 2000번~ 테스트"

# train_X = X.iloc[:2000]
# train_y = y.iloc[:2000]
# valid_X = X.iloc[2000:]
# valid_y = y.iloc[2000:]

# print(f"학습 데이터: {train_X.shape}")
# print(f"검증 데이터: {valid_X.shape}")

## 5. 평가 함수 정의 (⭐ 기출 대응!)

In [ ]:
def evaluate_classification(y_true, y_pred, y_pred_proba=None, model_name="Model"):
    """
    분류 모델 평가 - 모든 기출 지표 포함!
    
    기출 지표:
    - Accuracy: 2021년
    - F1 Score: 2022년, 2024년
    - ROC AUC: 2021년, 2022년, 2023년
    """
    acc = accuracy_score(y_true, y_pred)
    
    # 이진 분류 vs 다중 분류
    if len(np.unique(y_true)) == 2:
        # 이진 분류
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        
        print(f"\n{'='*60}")
        print(f"📊 {model_name} 성과 (이진 분류)")
        print(f"{'='*60}")
        print(f"Accuracy:  {acc:.4f}  ← 2021년 기출")
        print(f"Precision: {precision:.4f}")
        print(f"Recall:    {recall:.4f}")
        print(f"F1 Score:  {f1:.4f}  ← 2022, 2024년 기출")
        
        if y_pred_proba is not None:
            auc = roc_auc_score(y_true, y_pred_proba)
            print(f"ROC AUC:   {auc:.4f}  ← 2021, 2022, 2023년 기출")
        print(f"{'='*60}")
    else:
        # 다중 분류
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        f1 = f1_score(y_true, y_pred, average='weighted')
        
        print(f"\n{'='*60}")
        print(f"📊 {model_name} 성과 (다중 분류)")
        print(f"{'='*60}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"Precision (weighted): {precision:.4f}")
        print(f"Recall (weighted):    {recall:.4f}")
        print(f"F1 Score (weighted):  {f1:.4f}")
        print(f"{'='*60}")
    
    return {'Accuracy': acc, 'Precision': precision, 'Recall': recall, 'F1': f1}

## 6. Decision Tree - 기본 모델 (Full Tree)

In [ ]:
print("="*60)
print("🌳 Decision Tree - 기본 모델 (제약 없음)")
print("="*60)

# 모델 생성 (criterion은 'gini' 또는 'entropy')
tree_full = DecisionTreeClassifier(criterion='entropy', random_state=1)
tree_full.fit(train_X, train_y)

# 예측
pred_train_y = tree_full.predict(train_X)
pred_valid_y = tree_full.predict(valid_X)

# 확률 예측 (ROC AUC용)
if len(np.unique(y)) == 2:
    pred_valid_proba = tree_full.predict_proba(valid_X)[:, 1]
else:
    pred_valid_proba = None

# 성과 측정
print("\n[학습 데이터 성과]")
if HAS_DMBA:
    classificationSummary(train_y, pred_train_y, class_names=tree_full.classes_)
else:
    evaluate_classification(train_y, pred_train_y, model_name="Full Tree - Train")

print("\n[검증 데이터 성과]")
if HAS_DMBA:
    classificationSummary(valid_y, pred_valid_y, class_names=tree_full.classes_)
else:
    evaluate_classification(valid_y, pred_valid_y, pred_valid_proba, "Full Tree - Valid")

In [ ]:
# 나무 구조 정보
print("\n[나무 구조 정보]")
print(f"최대 깊이: {tree_full.get_depth()}")
print(f"리프 노드 수: {tree_full.get_n_leaves()}")
print(f"총 노드 수: {tree_full.tree_.node_count}")

In [ ]:
# 🎯 기출 대응: 다양한 평가 지표로 교차검증
print("\n" + "="*60)
print("📊 5-Fold 교차검증 (다양한 평가 지표)")
print("="*60)

# Accuracy
scores_acc = cross_val_score(tree_full, X, y, cv=5, scoring='accuracy')
print(f"\n✅ 5-Fold CV Accuracy: {scores_acc.mean():.4f} (±{scores_acc.std():.4f})")

# F1 Score
if len(np.unique(y)) == 2:
    scores_f1 = cross_val_score(tree_full, X, y, cv=5, scoring='f1')
    print(f"✅ 5-Fold CV F1 Score: {scores_f1.mean():.4f} (±{scores_f1.std():.4f})")
else:
    scores_f1 = cross_val_score(tree_full, X, y, cv=5, scoring='f1_weighted')
    print(f"✅ 5-Fold CV F1 Score (weighted): {scores_f1.mean():.4f} (±{scores_f1.std():.4f})")

# ROC AUC (이진 분류인 경우)
if len(np.unique(y)) == 2:
    scores_auc = cross_val_score(tree_full, X, y, cv=5, scoring='roc_auc')
    print(f"✅ 5-Fold CV ROC AUC: {scores_auc.mean():.4f} (±{scores_auc.std():.4f})")

## 7. 변수 중요도 분석

In [ ]:
# DataFrame으로 정리
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': tree_full.feature_importances_
}).sort_values('importance', ascending=False)

print("\n[변수 중요도 Top 10]")
print(importance_df.head(10).to_string(index=False))

In [ ]:
# 시각화
plt.figure(figsize=(10, 8))
top_features = importance_df.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance', fontsize=12)
plt.title('Top 15 Feature Importances', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 8. max_depth 튜닝 및 시각화 ⭐

In [ ]:
# 기출에서 자주 쓰는 평가 지표 선택
if len(np.unique(y)) == 2:
    scoring = 'roc_auc'  # 이진 분류: ROC AUC (2021, 2022, 2023)
    metric_name = 'ROC AUC'
else:
    scoring = 'accuracy'  # 다중 분류: Accuracy
    metric_name = 'Accuracy'

print(f"\n평가 지표: {metric_name}")
print("="*60)

In [ ]:
depths = range(2, 21)
train_scores = []
valid_scores = []
cv_scores = []

for depth in depths:
    tree = DecisionTreeClassifier(max_depth=depth, criterion='entropy', random_state=1)
    tree.fit(train_X, train_y)
    
    # 학습 데이터 점수
    if scoring == 'roc_auc':
        train_pred_prob = tree.predict_proba(train_X)[:, 1]
        valid_pred_prob = tree.predict_proba(valid_X)[:, 1]
        train_scores.append(roc_auc_score(train_y, train_pred_prob))
        valid_scores.append(roc_auc_score(valid_y, valid_pred_prob))
    else:
        train_scores.append(accuracy_score(train_y, tree.predict(train_X)))
        valid_scores.append(accuracy_score(valid_y, tree.predict(valid_X)))
    
    # 교차검증
    scores = cross_val_score(tree, X, y, cv=5, scoring=scoring)
    cv_scores.append(scores.mean())

# 최적 depth 찾기
best_depth_idx = np.argmax(cv_scores)
best_depth = list(depths)[best_depth_idx]
best_score = cv_scores[best_depth_idx]

print(f"\n✅ 최적 max_depth: {best_depth}")
print(f"✅ 최고 CV {metric_name}: {best_score:.4f}")

In [ ]:
# 시각화
plt.figure(figsize=(12, 6))
plt.plot(depths, train_scores, 'o-', label='Train', linewidth=2, markersize=6)
plt.plot(depths, valid_scores, 's-', label='Valid', linewidth=2, markersize=6)
plt.plot(depths, cv_scores, '^-', label='5-Fold CV', linewidth=2, markersize=6)
plt.axvline(best_depth, color='r', linestyle='--', linewidth=2, label=f'Best depth={best_depth}')
plt.xlabel('max_depth', fontsize=12, fontweight='bold')
plt.ylabel(metric_name, fontsize=12, fontweight='bold')
plt.title(f'Model Performance vs max_depth ({metric_name})', fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. 최적 모델 학습

In [ ]:
print("="*60)
print(f"🏆 최적 모델 (max_depth={best_depth})")
print("="*60)

tree_best = DecisionTreeClassifier(
    max_depth=best_depth,
    criterion='entropy',
    random_state=1
)
tree_best.fit(train_X, train_y)

# 예측
pred_train_y = tree_best.predict(train_X)
pred_valid_y = tree_best.predict(valid_X)

# 확률 예측
if len(np.unique(y)) == 2:
    pred_valid_proba = tree_best.predict_proba(valid_X)[:, 1]
else:
    pred_valid_proba = None

print("\n[학습 데이터 성과]")
if HAS_DMBA:
    classificationSummary(train_y, pred_train_y, class_names=tree_best.classes_)
else:
    evaluate_classification(train_y, pred_train_y, model_name="Best Tree - Train")

print("\n[검증 데이터 성과]")
if HAS_DMBA:
    classificationSummary(valid_y, pred_valid_y, class_names=tree_best.classes_)
else:
    best_results = evaluate_classification(valid_y, pred_valid_y, pred_valid_proba, "Best Tree - Valid")

In [ ]:
# 🎯 기출 대응: 모든 주요 지표 출력
print("\n" + "="*60)
print("📊 최적 모델 교차검증 성과 (모든 지표)")
print("="*60)

# Accuracy
scores_acc_best = cross_val_score(tree_best, X, y, cv=5, scoring='accuracy')
print(f"\n✅ 5-Fold CV Accuracy: {scores_acc_best.mean():.4f} (±{scores_acc_best.std():.4f})")

# F1 Score
if len(np.unique(y)) == 2:
    scores_f1_best = cross_val_score(tree_best, X, y, cv=5, scoring='f1')
    print(f"✅ 5-Fold CV F1 Score: {scores_f1_best.mean():.4f} (±{scores_f1_best.std():.4f})")
else:
    scores_f1_best = cross_val_score(tree_best, X, y, cv=5, scoring='f1_weighted')
    print(f"✅ 5-Fold CV F1 Score (weighted): {scores_f1_best.mean():.4f} (±{scores_f1_best.std():.4f})")

# ROC AUC (이진 분류)
if len(np.unique(y)) == 2:
    scores_auc_best = cross_val_score(tree_best, X, y, cv=5, scoring='roc_auc')
    print(f"✅ 5-Fold CV ROC AUC: {scores_auc_best.mean():.4f} (±{scores_auc_best.std():.4f})")

In [ ]:
# 최적 모델의 변수 중요도
importance_best = pd.DataFrame({
    'feature': X.columns,
    'importance': tree_best.feature_importances_
}).sort_values('importance', ascending=False)

print("\n[최적 모델의 변수 중요도 Top 10]")
print(importance_best.head(10).to_string(index=False))

## 10. GridSearchCV로 종합 튜닝 (선택사항)

In [ ]:
# 여러 하이퍼파라미터 동시 튜닝
param_grid = {
    'max_depth': list(range(2, 16)),
    'min_samples_split': [10, 15, 20],
    'min_samples_leaf': [5, 10, 15]
}

grid_search = GridSearchCV(
    DecisionTreeClassifier(criterion='entropy', random_state=1),
    param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=1
)

print("\n⏳ GridSearchCV 진행 중...")
grid_search.fit(train_X, train_y)

print(f"\n✅ 최적 파라미터: {grid_search.best_params_}")
print(f"✅ 최고 CV {metric_name}: {grid_search.best_score_:.4f}")

# 최적 모델
tree_grid = grid_search.best_estimator_

## 11. 로지스틱 회귀와 비교 (문제에서 요구시) ⭐

In [ ]:
# 기출: Top 10 또는 Top 5 중요 변수로 로지스틱 회귀
top_n_features = 10  # 또는 5 (기출에 따라 변경)
top_features = importance_best.head(top_n_features)['feature'].tolist()

print(f"\nTop {top_n_features} 중요 변수:")
print(top_features)

logreg = LogisticRegression(random_state=42, solver='liblinear', max_iter=1000)

# 교차검증
print("\n" + "="*60)
print(f"📊 로지스틱 회귀 (Top {top_n_features} 변수) 교차검증")
print("="*60)

# Accuracy
scores_logreg_acc = cross_val_score(logreg, X[top_features], y, cv=5, scoring='accuracy')
print(f"\n✅ 5-Fold CV Accuracy: {scores_logreg_acc.mean():.4f} (±{scores_logreg_acc.std():.4f})")

# F1 Score
if len(np.unique(y)) == 2:
    scores_logreg_f1 = cross_val_score(logreg, X[top_features], y, cv=5, scoring='f1')
    print(f"✅ 5-Fold CV F1 Score: {scores_logreg_f1.mean():.4f} (±{scores_logreg_f1.std():.4f})")
else:
    scores_logreg_f1 = cross_val_score(logreg, X[top_features], y, cv=5, scoring='f1_weighted')
    print(f"✅ 5-Fold CV F1 Score (weighted): {scores_logreg_f1.mean():.4f} (±{scores_logreg_f1.std():.4f})")

# ROC AUC
if len(np.unique(y)) == 2:
    scores_logreg_auc = cross_val_score(logreg, X[top_features], y, cv=5, scoring='roc_auc')
    print(f"✅ 5-Fold CV ROC AUC: {scores_logreg_auc.mean():.4f} (±{scores_logreg_auc.std():.4f})")

In [ ]:
# 비교 (기출 스타일)
print("\n" + "="*60)
print(f"🏆 모델 비교 ({metric_name} 기준)")
print("="*60)

if scoring == 'roc_auc':
    tree_score = scores_auc_best.mean()
    logreg_score = scores_logreg_auc.mean()
elif scoring == 'f1':
    tree_score = scores_f1_best.mean()
    logreg_score = scores_logreg_f1.mean()
else:
    tree_score = scores_acc_best.mean()
    logreg_score = scores_logreg_acc.mean()

print(f"\n의사결정트리 (max_depth={best_depth}): {tree_score:.4f}")
print(f"로지스틱 회귀 (Top {top_n_features} 변수):  {logreg_score:.4f}")

if tree_score > logreg_score:
    print(f"\n✅ 의사결정트리가 더 우수 (차이: {tree_score - logreg_score:.4f})")
else:
    print(f"\n✅ 로지스틱 회귀가 더 우수 (차이: {logreg_score - tree_score:.4f})")

## 12. 최종 성과 및 시각화

In [ ]:
# 이진 분류인 경우 ROC AUC 계산 및 시각화
if len(np.unique(y)) == 2:
    valid_pred_prob = tree_best.predict_proba(valid_X)[:, 1]
    auc_score = roc_auc_score(valid_y, valid_pred_prob)
    
    print("\n" + "="*60)
    print("📊 ROC Curve")
    print("="*60)
    print(f"\n최적 의사결정트리 ROC AUC (검증): {auc_score:.4f}")
    
    # ROC 곡선
    fpr, tpr, thresholds = roc_curve(valid_y, valid_pred_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC curve (AUC = {auc_score:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curve - Best Decision Tree', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("\n다중 분류: ROC 곡선은 이진 분류에서만 사용됩니다.")
    print("대신 Confusion Matrix를 확인하세요.")

In [ ]:
# Confusion Matrix
print("\n" + "="*60)
print("🔍 Confusion Matrix")
print("="*60)

cm = confusion_matrix(valid_y, pred_valid_y)
print("\n", cm)

# 시각화
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.title('Confusion Matrix - Best Decision Tree', fontsize=14, fontweight='bold')
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Classification Report
print("\n" + "="*60)
print("📋 Classification Report")
print("="*60)
print("\n", classification_report(valid_y, pred_valid_y))

## 13. 최종 결과 요약 ✅

In [ ]:
print("="*80)
print("🎯 최종 결과 요약 (기출 대응 완료!)")
print("="*80)

print(f"""
✅ 분석 완료!

🌳 최적 의사결정트리:
   - max_depth: {best_depth}
   - criterion: entropy
   - 나무 깊이: {tree_best.get_depth()}
   - 리프 노드 수: {tree_best.get_n_leaves()}

📊 검증 데이터 성과:
   - Accuracy:  {best_results['Accuracy']:.4f}  ← 2021년 기출 지표
   - F1 Score:  {best_results['F1']:.4f}  ← 2022, 2024년 기출 지표
   - Precision: {best_results['Precision']:.4f}
   - Recall:    {best_results['Recall']:.4f}
""")

if len(np.unique(y)) == 2:
    print(f"   - ROC AUC:   {auc_score:.4f}  ← 2021, 2022, 2023년 기출 지표")

print(f"""
📊 5-Fold CV 성과:
   - Accuracy:  {scores_acc_best.mean():.4f} (±{scores_acc_best.std():.4f})
   - F1 Score:  {scores_f1_best.mean():.4f} (±{scores_f1_best.std():.4f})
""")

if len(np.unique(y)) == 2:
    print(f"   - ROC AUC:   {scores_auc_best.mean():.4f} (±{scores_auc_best.std():.4f})")

print(f"""
🔝 중요 변수 Top 5:
   {importance_best.head(5)['feature'].tolist()}

🎉 모든 기출 유형 대응 완료!
""")